# Interacting with the ESCALATE REST API

This is a tutorail for using the ESCALATE REST API. 

It presumes only knowledge of basic python. 

It also presumes you have a local instance of ESCALATE running at http://localhost:8000

In [1]:
import json
from pprint import pprint

import requests  # requests library will send and receive data from the escalate server
from requests.api import post

import pandas as pd

## Quick intro to REST APIs

All you need to know about REST is that:
* It is a protocol for exchanging data between a client (e.g. me) and a server (e.g. ESCALATE)
* Data formats are human *and* machine readable (XML or JSON)
* There are two main HTTP 'verbs' or functions: 
  1. GET data from the server
  2. POST data to the server  
     (there are others, e.g. PUT and PATCH, but we won't use these in this tutorial)
* The python `requests` library implements the HTTP verbs for interacting with servers, including with REST APIs

### Example: GET from the PubChem API

* [Pubchem](https://pubchem.ncbi.nlm.nih.gov/) is a great way to get chemical informatics data from a vast repository
   - e.g. just search for any compound above
   - Has a graphical web interface and REST API
* (There are other computational chemistry REST APIs, including [Open Chemistry](https://doi.org/10.1186/s13321-017-0241-z), and [AFLOW-ML](https://doi.org/10.1016/j.commatsci.2018.03.075))


In [2]:
# dict mapping compound name to PubChemID
PubChemIDs = {
    'Methane': '297',
    'Benzene': '241',
    'Ethylammonium Iodide': '11116533'
}

The URL below is the compound 'API endpoint' which I can send a request to for properties of compounds given their PubChem CIDs

In [3]:
response = requests.get(('https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound'          # What type of entity? Compound
                         f'/cid/{",".join(PubChemIDs.values())}/'                      # Which compounds? These
                         'property/MolecularFormula,MolecularWeight,CanonicalSMILES/'  # Which properties? 
                         'JSON'))                                                     # Which format?
                        #proxies=proxies)

In [4]:
response

<Response [200]>

In [5]:
response.json()

{'PropertyTable': {'Properties': [{'CID': 297,
    'MolecularFormula': 'CH4',
    'MolecularWeight': 16.043,
    'CanonicalSMILES': 'C'},
   {'CID': 241,
    'MolecularFormula': 'C6H6',
    'MolecularWeight': 78.11,
    'CanonicalSMILES': 'C1=CC=CC=C1'},
   {'CID': 11116533,
    'MolecularFormula': 'C2H8IN',
    'MolecularWeight': 173,
    'CanonicalSMILES': 'CCN.I'}]}}

## The ESCALATE REST API 

* Send data back and forth between ML, chemists, and laboratories

### Interactive List of all available ESCALATE endpoints

Simply navigating to http://localhost:8000/api will show you a browsable list of all available API endpoints. 

We can also view that list programmatically: 

In [6]:
base_url = 'http://localhost:8000/api'  # local dev server
response = requests.get(base_url)
response.json()

{'action': 'http://localhost:8000/api/action/',
 'actiondef': 'http://localhost:8000/api/actiondef/',
 'actionparameter': 'http://localhost:8000/api/actionparameter/',
 'actionparameterdefassign': 'http://localhost:8000/api/actionparameterdefassign/',
 'actor': 'http://localhost:8000/api/actor/',
 'billofmaterials': 'http://localhost:8000/api/billofmaterials/',
 'bomcompositematerial': 'http://localhost:8000/api/bomcompositematerial/',
 'bommaterial': 'http://localhost:8000/api/bommaterial/',
 'calculation': 'http://localhost:8000/api/calculation/',
 'calculationdef': 'http://localhost:8000/api/calculationdef/',
 'compositematerial': 'http://localhost:8000/api/compositematerial/',
 'compositematerialproperty': 'http://localhost:8000/api/compositematerialproperty/',
 'condition': 'http://localhost:8000/api/condition/',
 'conditioncalculationdefassign': 'http://localhost:8000/api/conditioncalculationdefassign/',
 'conditiondef': 'http://localhost:8000/api/conditiondef/',
 'experiment': '

### Logging into ESCALATE


To be able to view and post to all endpoints, I will create a log in session that is managed by a token. 

In [7]:
# demo login credentials
login_data = {
    'username': 'mtynes',
    'password': 'hello1world2'
}

r_login = requests.post(f'{base_url}/login', data=login_data)
token = r_login.json()['token']
token

'558bd01df8969e1fb6d2b25a77ea0c566d0fe12b'

In [8]:
token_header = {'Authorization': f'Token {token}'}

content_type_header = {'content-type': 'application/json'} # for most requests we'll want this header

This token will allow me to validate my identity for this session

### Simple helper functions for GET/POST

These functions will do some minimal URL generation and JSON response parsing on top of what is done by the `requests` library

In [9]:
def post_data(endpoint, data={}, headers={**token_header, **content_type_header}):
    """POST `data` to `endpoint`in ESCALATE API using `headers`
    
    return: (dict|requests.Response), bool
    """
    r = requests.post(f'{base_url}/{endpoint}/', 
                      data=json.dumps(data), 
                      headers=headers)
    print(r)
    if r.ok: 
        print('POST: OK, returning new resource dict')
        return r.json()
    print('POST: FAILED, returning response object')
    return r


def get_data(endpoint, data={}, headers={**token_header}):
    """Make GET request with `data` to `endpoint` in ESCALATE API using `headers`
    
    return: (dict|list|requests.Response), bool
    """
    r = requests.get(f'{base_url}/{endpoint}/', params=data, headers=headers)
    print(r)
    if r.ok: 
        print('GET: OK')
        
        resp_json = r.json()        
        
        # handle cases: one vs many results
        if resp_json.get('count') is None: # edge case: template edit
            return r.json()
        elif resp_json.get('count') == 1: 
            print('Found one resource, returning dict')
            return resp_json['results'][0]
        elif resp_json.get('count') >= 1: 
            print(f"Found {resp_json['count']} resources, returning list of dicts)")
            return r.json()['results']
        else:
            print('GET: FAILED, returning response object')
    return r

### GET all of materials defined in ESCALATE

In [10]:
r = get_data(endpoint='material')

<Response [200]>
GET: OK
Found 294 resources, returning list of dicts)


All of these materials would probably take up too much space in this notebook, lets look at the first one.

All lead iodide shares in this chemical formula, etc. Models are jsut 'what is is about these thigns that are all the same, as opposed to particular vials of gblthat has volumes, contanimanants, dates, masses. these all come from the object, but share in model. models dont contain provenance, are only abotu intenseive properties 

In [11]:
r[0]

{'url': 'http://localhost:8000/api/material/9d2e43b5-7874-4de4-ab24-b554ea3a4aa4/',
 'uuid': '9d2e43b5-7874-4de4-ab24-b554ea3a4aa4',
 'edocs': [],
 'tags': [],
 'notes': [],
 'description': 'Gamma-Butyrolactone',
 'consumable': True,
 'composite_flg': False,
 'material_class': 'model',
 'actor_description': None,
 'status_description': 'active',
 'add_date': '2021-05-03T15:37:53.096175Z',
 'mod_date': '2021-05-03T15:38:01.237109Z',
 'actor': None,
 'status': 'http://localhost:8000/api/status/88cc1f22-5e0e-4707-b7f1-235730c29bb2/',
 'material_types': ['http://localhost:8000/api/materialtype/9dfe9596-8264-4307-af4c-362adcc6bc2a/',
  'http://localhost:8000/api/materialtype/e110ca12-d7b3-4ee9-b1b3-b9b683c0a69b/',
  'http://localhost:8000/api/materialtype/424cce16-3405-43d7-b9ff-64171906500f/'],
 'property': []}

These are the fields available for materials. Notably we can associate a material with arbitrary properties and material types.

### POST a material property

#### Current property definitions in ESCALATE

ESCALATE supports user defined property definitions, these are the one's we're using

In [12]:
r = get_data('propertydef', 
             {'fields': ['description']} # we can select 'columns' with the fields parameter    
            )
r

<Response [200]>
GET: OK
Found 16 resources, returning list of dicts)


[{'description': 'particle-size {min, max}'},
 {'description': 'mesh {min, max}'},
 {'description': 'capacity'},
 {'description': 'cross-linkage %'},
 {'description': 'moisture % {min, max}'},
 {'description': 'Resin Class'},
 {'description': 'Functional group'},
 {'description': 'concentration_molarity'},
 {'description': 'manufacturer'},
 {'description': 'plate well count'},
 {'description': 'plate well location'},
 {'description': 'plate well robot order'},
 {'description': 'plate well volume {min, max}'},
 {'description': 'concentration'},
 {'description': 'molecular-weight'},
 {'description': 'Crystal Score'}]

#### Molecular Weight

In [13]:
mw_property_def = get_data('propertydef', 
                           {'description': 'molecular-weight', # find an entity that has a particular description
                            'fields': ['url', 'description', 'val_unit']})
mw_property_def

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/propertydef/c47df850-d65a-40f3-b97e-bfaed9bf3435/',
 'description': 'molecular-weight',
 'val_unit': 'g/mol'}

Now we can associate an instance of this property with a material, say EthNH3I

In [14]:
ethylammonium_iodide = get_data('material', 
                                {'description': 'Ethylammonium Iodide',
                                 'fields':['url', 'description', 'property']})
ethylammonium_iodide

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/4d90b104-c81d-4915-8307-97349af873bf/',
 'description': 'Ethylammonium Iodide',
 'property': ['http://localhost:8000/api/property/b52f4930-1b98-4f17-a58d-7e24881adf5e/']}

Note the empty property list above

Lets fill that in with the molecular weight from PubChem

In [15]:
response = requests.get('https://pubchem.ncbi.nlm.nih.gov/rest/pug/'+\
                        f'compound/cid/{PubChemIDs.get("Ethylammonium Iodide")}/'+\
                        'property/MolecularFormula,MolecularWeight,CanonicalSMILES/JSON')
                      # proxies=proxies)
pubchem_json = response.json()
eth_mw = pubchem_json['PropertyTable']['Properties'][0]['MolecularWeight']
eth_mw

173

In [16]:
r = post_data('materialproperty',
                  {'material': ethylammonium_iodide['url'],
                   'property_def': mw_property_def['url'],
                   'value': f"{eth_mw}"
                  }
                 )

<Response [201]>
POST: OK, returning new resource dict


And we've stored it!

In [17]:
get_data('material', 
         {'description':'Ethylammonium Iodide'})

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/material/4d90b104-c81d-4915-8307-97349af873bf/',
 'uuid': '4d90b104-c81d-4915-8307-97349af873bf',
 'edocs': [],
 'tags': [],
 'notes': [],
 'description': 'Ethylammonium Iodide',
 'consumable': True,
 'composite_flg': False,
 'material_class': 'model',
 'actor_description': None,
 'status_description': 'active',
 'add_date': '2021-05-03T15:37:53.096175Z',
 'mod_date': '2021-05-03T15:38:01.237109Z',
 'actor': None,
 'status': 'http://localhost:8000/api/status/88cc1f22-5e0e-4707-b7f1-235730c29bb2/',
 'material_types': ['http://localhost:8000/api/materialtype/9dfe9596-8264-4307-af4c-362adcc6bc2a/',
  'http://localhost:8000/api/materialtype/e110ca12-d7b3-4ee9-b1b3-b9b683c0a69b/'],
 'property': ['http://localhost:8000/api/property/b52f4930-1b98-4f17-a58d-7e24881adf5e/',
  'http://localhost:8000/api/property/5d6a66ed-a0b0-45b7-b164-129c2d3588ef/']}


* In practice we can use this functionality to store properties from any experiment or calculation. 

* We can also store metadata about where these values came from   
  (example to come in a further tutorial on tags, notes, edocs, calculations).

## Action definitions

* Just as we are free to define properties, we are free to define actions
* Current definition are what we've needed to specify human/robot instructions for current workflows

In [18]:
r = get_data('actiondef', {'fields': ['description', 'uuid', 'url']})
r

<Response [200]>
GET: OK
Found 7 resources, returning list of dicts)


[{'url': 'http://localhost:8000/api/actiondef/be6c1bd1-45e0-4221-9078-36c72fbcc978/',
  'uuid': 'be6c1bd1-45e0-4221-9078-36c72fbcc978',
  'description': 'dispense'},
 {'url': 'http://localhost:8000/api/actiondef/1c9318bf-aef1-4724-85b7-e4bc138836f2/',
  'uuid': '1c9318bf-aef1-4724-85b7-e4bc138836f2',
  'description': 'heat_stir'},
 {'url': 'http://localhost:8000/api/actiondef/50652482-9b8b-411c-bf99-c000418b8872/',
  'uuid': '50652482-9b8b-411c-bf99-c000418b8872',
  'description': 'heat'},
 {'url': 'http://localhost:8000/api/actiondef/e7b5a3ab-dedc-41f1-92ed-0d21fadfbc7e/',
  'uuid': 'e7b5a3ab-dedc-41f1-92ed-0d21fadfbc7e',
  'description': 'start_node'},
 {'url': 'http://localhost:8000/api/actiondef/f09cf7c5-547b-4ade-9517-df809c0c8454/',
  'uuid': 'f09cf7c5-547b-4ade-9517-df809c0c8454',
  'description': 'end_node'},
 {'url': 'http://localhost:8000/api/actiondef/86734a65-99fa-4383-ac81-e8d22c95c916/',
  'uuid': '86734a65-99fa-4383-ac81-e8d22c95c916',
  'description': 'dispense_solid'},

#### Zooming in on the dispense action definition

In [19]:
get_data('actiondef',               
         {'description': 'dispense',  # which action def
          'expand': 'parameter_def',  # sub dictionary to expand
          })

<Response [200]>
GET: OK
Found one resource, returning dict


{'url': 'http://localhost:8000/api/actiondef/be6c1bd1-45e0-4221-9078-36c72fbcc978/',
 'uuid': 'be6c1bd1-45e0-4221-9078-36c72fbcc978',
 'edocs': [],
 'tags': [],
 'notes': [{'url': 'http://localhost:8000/api/note/3bc81cfd-e2e7-431d-a03e-7cd72cde2e67/',
   'uuid': '3bc81cfd-e2e7-431d-a03e-7cd72cde2e67',
   'notetext': 'source material = material to be dispensed, destination material = plate well',
   'add_date': '2021-05-03T15:37:58.233458Z',
   'mod_date': '2021-05-03T15:37:58.233458Z',
   'actor_description': 'Mike Tynes',
   'ref_note_uuid': 'be6c1bd1-45e0-4221-9078-36c72fbcc978',
   'actor': 'http://localhost:8000/api/actor/d9b24fd8-b799-43b3-9fe1-c242edd755c8/'}],
 'parameter_def': [{'url': 'http://localhost:8000/api/parameterdef/0f13f48f-e818-4cb2-b4b9-e18398eed028/',
   'uuid': '0f13f48f-e818-4cb2-b4b9-e18398eed028',
   'edocs': [],
   'tags': [],
   'notes': [],
   'description': 'volume',
   'default_val': {'value': 0.0, 'unit': 'mL', 'type': 'num'},
   'unit_type': 'volume',
  

### Actions + Materials = Experiment Template 

* Experiment template = the form of an experiment that I wish to re-use, varying material choices and process parameters

In [20]:
experiment_templates = get_data('experimenttemplate', 
                               {'fields':['description', 'url']})

<Response [200]>
GET: OK
Found 4 resources, returning list of dicts)


In [21]:
experiment_templates

[{'url': 'http://localhost:8000/api/experimenttemplate/2ffd9f1d-5d92-4581-abf8-5eb68e7a1ed7/',
  'description': 'liquid_solid_extraction'},
 {'url': 'http://localhost:8000/api/experimenttemplate/669dc3da-777d-4266-b453-cc88fb3c6b07/',
  'description': 'resin_weighing'},
 {'url': 'http://localhost:8000/api/experimenttemplate/01aad3de-4179-4caf-bcc0-48b454190064/',
  'description': 'perovskite_demo'},
 {'url': 'http://localhost:8000/api/experimenttemplate/fb8cf185-7ebb-4e88-b6ee-f1ea2b25e032/',
  'description': 'test_wf_1'}]

Click on perovskite demo link and note 2 main nested fields: 
* Bill of materials = the initial materials for the perovskite workflow
* Workflow = the set of actions that combine these materials into perovskite crystal trials

In [22]:
perovskite_template = get_data('experimenttemplate',
                              {'description': 'test_wf_1', 
                              'expand': 'workflow' # expand the workflow subdictionary
                              })

<Response [200]>
GET: OK
Found one resource, returning dict


In [23]:
perovskite_template

{'url': 'http://localhost:8000/api/experimenttemplate/fb8cf185-7ebb-4e88-b6ee-f1ea2b25e032/',
 'uuid': 'fb8cf185-7ebb-4e88-b6ee-f1ea2b25e032',
 'edocs': [],
 'tags': [],
 'notes': [],
 'bill_of_materials': [{'url': 'http://localhost:8000/api/billofmaterials/b66df4d3-ed02-4965-943d-35e4ffab3ee8/',
   'uuid': 'b66df4d3-ed02-4965-943d-35e4ffab3ee8',
   'edocs': [],
   'tags': [],
   'notes': [],
   'bom_material': ['http://localhost:8000/api/bommaterial/5bd3674b-2d52-4049-97e8-bde6ff7a32b3/',
    'http://localhost:8000/api/bommaterial/fdcbf63a-5d92-46e7-8728-001ec94b30ea/',
    'http://localhost:8000/api/bommaterial/08b17acd-22a1-4b61-9a1a-fd3fec4197df/',
    'http://localhost:8000/api/bommaterial/b607708d-bc91-4c79-8dac-c60038b07d92/',
    'http://localhost:8000/api/bommaterial/8a0467cf-8be7-49c7-b584-f45ae7c3b36d/'],
   'description': 'Test WF1 Materials',
   'experiment_description': 'test_wf_1',
   'add_date': '2021-05-03T15:38:01.377484Z',
   'mod_date': '2021-05-03T15:38:01.377484Z'

## Bill of Materials

The initial materials: think list of materials in methods section of paper

These are the Bill of Materials' Material entries for this experiment: 

In [24]:
get_data('bommaterial', {'bom':perovskite_template['bill_of_materials'][0]['uuid'], 
                        'fields':['description']})

<Response [200]>
GET: OK
Found 5 resources, returning list of dicts)


[{'description': 'Stock B'},
 {'description': 'Solvent'},
 {'description': 'Plate'},
 {'description': 'Stock A'},
 {'description': 'Acid'}]

In [25]:
get_data('compositematerial', {'composite_description__startswith':'Stock', 
                               'fields': ['composite_description',
                                          'component_description']})

<Response [200]>
GET: OK
Found 7 resources, returning list of dicts)


[{'composite_description': 'Stock A',
  'component_description': 'Lead Diiodide'},
 {'composite_description': 'Stock A',
  'component_description': 'Ethylammonium Iodide'},
 {'composite_description': 'Stock A',
  'component_description': 'Gamma-Butyrolactone'},
 {'composite_description': 'Stock B',
  'component_description': 'Ethylammonium Iodide'},
 {'composite_description': 'Stock B',
  'component_description': 'Gamma-Butyrolactone'},
 {'composite_description': 'Stock FAH',
  'component_description': 'Formic Acid'},
 {'composite_description': 'Stock FAH', 'component_description': 'Water'}]

#### We could drill further to get the concentrations, properties, etc

### Workflows: Logical groups of actions

Each of these contains a set of parameters that we can edit

In [26]:
perovskite_demo = get_data('experimenttemplate',
         {'description': 'perovskite_demo',
          'expand': 'workflow'})
[wf['description'] for wf in perovskite_demo['workflow']]

<Response [200]>
GET: OK
Found one resource, returning dict


['Perovskite Demo: Preheat Plate',
 'Perovskite Demo: Prepare Stock A',
 'Perovskite Demo: Prepare Stock B',
 'Perovskite Demo: Dispense Solvent',
 'Perovskite Demo: Dispense Stock A',
 'Perovskite Demo: Dispense Stock B',
 'Perovskite Demo: Dispense Acid Vol 1',
 'Perovskite Demo: Heat Stir 1',
 'Perovskite Demo: Dispense Acid Vol 2',
 'Perovskite Demo: Heat Stir 2',
 'Perovskite Demo: Heat']

In [27]:
dispense_solvent_wf = perovskite_demo['workflow'][3] # pull out a workflow
example_steps = dispense_solvent_wf['step'][4]     # pull out some steps
example_steps

{'url': 'http://localhost:8000/api/workflowstep/25725c7b-269f-4bdc-90d9-7a875e2dff87/',
 'uuid': '25725c7b-269f-4bdc-90d9-7a875e2dff87',
 'edocs': [],
 'tags': [],
 'notes': [],
 'workflow_object': 'http://localhost:8000/api/workflowobject/61a44486-f75f-46a1-ae4c-4e64b84bcaf0/',
 'workflow_description': 'Perovskite Demo: Dispense Solvent',
 'parent_object_type': 'action',
 'parent_object_description': 'Perovskite Demo: Dispense Solvent: Solvent -> Plate: 96 Well Plate well#: A4',
 'parent_path': None,
 'conditional_val': None,
 'conditional_value': None,
 'status_description': None,
 'add_date': '2021-05-03T15:38:03.802814Z',
 'mod_date': '2021-05-03T15:38:03.802814Z',
 'workflow': 'http://localhost:8000/api/workflow/cdb3791d-1cfc-4386-b516-f5bb91f0b974/',
 'parent': 'http://localhost:8000/api/workflowstep/69c17f15-78b1-4446-b116-aa094e408f96/',
 'status': None}

## Creating a new workflow from a template

If I want create an instance experiment from a template I: 

In [28]:
editable_template = get_data(
    # template endpoint  /        template ID      / create an instance of this template 
    f'experimenttemplate/{perovskite_demo["uuid"]}/create'
)

<Response [200]>
GET: OK


In [29]:
editable_template

{'experiment_name': '',
 'material_parameters': [{'material_name': 'Stock B',
   'value': 'http://localhost:8000/api/inventorymaterial/44a2a5a2-41df-4f96-abc9-7cea4668c79f/'},
  {'material_name': 'Solvent',
   'value': 'http://localhost:8000/api/inventorymaterial/6c88c051-e09c-420c-b0f5-391e8830311b/'},
  {'material_name': 'Organic',
   'value': 'http://localhost:8000/api/inventorymaterial/ad10fc51-af44-4db1-b63d-e3fe816f2ef9/'},
  {'material_name': 'Plate',
   'value': 'http://localhost:8000/api/inventorymaterial/a2f304e9-b23b-49fc-a927-b6297d67ec52/'},
  {'material_name': 'Stock A',
   'value': 'http://localhost:8000/api/inventorymaterial/44a2a5a2-41df-4f96-abc9-7cea4668c79f/'},
  {'material_name': 'Acid',
   'value': 'http://localhost:8000/api/inventorymaterial/4b7ff63b-fe11-4853-9344-f26e3f947ed8/'},
  {'material_name': 'Inorganic',
   'value': 'http://localhost:8000/api/inventorymaterial/fb04d115-84ce-4e76-82c3-9ef530dee4b8/'}],
 'experiment_parameters_1': [{'object_description': 

Can also give arrays of values over 96 well plates

In [30]:
editable_template['experiment_parameters_1'][0]['value']['value'] = 1.1

In [31]:
editable_template['material_parameters'][1]['value'] = 'http://localhost:8000/api/inventorymaterial/c2faef65-3d6b-43cb-8b8f-fd4c1c1e71aa/'

Todo: 


* .value.value is wierd 
* get parameters in the editable tempalte by name rather than index? 
* Write unit tests for 
    * q1, q2, q3, q1_material
    * right now it looks like only q1 works
    * q3 needs ad-hoc logic until calculation is fixed
    * q1_mat fails
    * q2 is untested

In [32]:
from IPython.display import JSON

In [33]:
JSON(editable_template)

<IPython.core.display.JSON object>

In [34]:
editable_template['experiment_name'] = 'test_perovskite_instance_updated'

Suppose I edit this json then I can post the new template to the server

In [36]:
%%time
resp = post_data(
    f'experimenttemplate/{perovskite_demo["uuid"]}/create',
    editable_template
)

<Response [200]>
POST: OK, returning new resource dict
CPU times: user 4.4 ms, sys: 2.69 ms, total: 7.09 ms
Wall time: 1min 5s


This is pretty slow for unknown reasons

In [37]:
resp

{'new_experiment_created': 'http://localhost:8000/api/experiment/7f9c6c49-3a5d-4ba7-896c-66d8e08d9259/'}

This experiment then appears in the experiment queue.   

http://localhost:8000/experiment_list

The experimentalist is notified, can download relevant robot input and upload observed values through forms

In [ ]:
%%time
new_experiment_json = get_data('experiment/' + resp['new_experiment_created'].split('/')[-2], 
                                {'expand': 'workflow.step.workflow_object.action.parameter'}) # expanding deeply nested fields is somewhat slow

In [ ]:
experiment_json = [new_experiment_json]

In [ ]:
experiment_json

In [ ]:
def experiment_json_to_df(experiment_json):
    result = []
    for e in experiment_json: 
        for workflow in e['workflow']:
            if 'Dispense' not in workflow['description']:
                    continue
            for step in workflow['step']: 
                action = step['workflow_object']['action']
                for parameter in action['parameter']:
                    result.append(
                        dict(
                            experiment_url         = e['url'],
                            experiment_id          = e['url'].split('/')[-2],
                            action_source          = action['source_material_description'],
                            action_dest            = action['destination_material_description'],
                            action_parameter       = parameter['parameter_def_description'],
                            action_parameter_value = float(parameter['parameter_val_nominal']['value']),
                            action_parameter_unit  = parameter['parameter_val_nominal']['unit']
                            )
                        )
    return pd.DataFrame(result)

In [ ]:
results = experiment_json_to_df(experiment_json)

In [ ]:
results

In [ ]:
results = results.pivot_table(index=['experiment_id', 'action_dest'], 
                             columns=['action_source'], 
                             values='action_parameter_value')
results

In [ ]:
crystal_scores = get_data('measure', 
                          {'measuredef':
                                (get_data('measuredef', 
                                          {'description': 'crystal_score'})['url']
                                )
                          })                        
results['crystal_score'] = crystal_scores['measure_value']['value']

In [ ]:
results

### Current Limitations

* Some parts of API still are 'high entropy' (e.g. measure)
* Ditto for some portions of UI
* REST is slow for large transfers